In [1]:
import pandas as pd
import pickle as pkl

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

import scipy
from scipy.stats import spearmanr
from scipy import sparse as sp

import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import cardio_mesh
from cardio_mesh import Cardiac3DMesh

from cardio_mesh.paths import (
    get_3d_pointcloud_file,
    get_4d_pointcloud
)

from cardio_mesh.lv_aha_quantification import (
    compute_thickness_per_aha
)

all_ids = cardio_mesh.list_mesh_ids()

In [3]:
meshes = dict( get_4d_pointcloud(all_ids[:10], return_id=True) )


In [5]:
meshes['2897041'].shape

(50, 19455, 3)

In [4]:
compute_thickness_per_aha(meshes['2897041'])

AssertionError: The input point cloud array should have 4396 points (corresponding to a left ventricle without the valve surfaces).

In [ ]:
dict( get_4d_pointcloud(all_ids[:10], return_id=True) )

___

### Wall thickness

For each point in the epicardial surface, find closest point in the endocardial surface.

In [ ]:
@interact
def count_vertices(aha_index=widgets.IntSlider(min=1, max=17)):
    print (sum((epi_endo_labels == EPI) & (lv_aha_labels == aha_index)))    

interactive(children=(IntSlider(value=1, description='aha_index', max=17, min=1), Output()), _dom_classes=('wi…

In [ ]:
%%timeit
np.array(compute_thickness_per_aha("1000215"))

UnboundLocalError: local variable 'point_cloud_array' referenced before assignment

In [ ]:
import multiprocessing

def worker_function(args):
    
    start, end = args
    
    for i in range(start, end):
        id = ids[i]
        NPY_FILE = f"notebooks/thicknesses/{id}_thickness_per_aha.npy"
        thickness = compute_thickness_per_aha(id)
        np.save(NPY_FILE, thickness)
        

def parallel_for_loop(num_cores, total_iterations):
    
    chunk_size = total_iterations // num_cores
    pool = multiprocessing.Pool(processes=num_cores)
    indices = [(i, i + chunk_size) for i in range(0, total_iterations, chunk_size)]
    pool.map(worker_function, indices)
    pool.close()
    pool.join()

In [ ]:
TOTAL_ITERATIONS = 60000
ids = [ 
    id for id in all_ids[:TOTAL_ITERATIONS] 
    if not os.path.exists(f"notebooks/thicknesses/{id}_thickness_per_aha.npy") 
]

print(len(ids))

NUM_CORES = 200 # multiprocessing.cpu_count()  # Use all available CPU cores
parallel_for_loop(NUM_CORES, TOTAL_ITERATIONS)
# result = parallel_for_loop(NUM_CORES, TOTAL_ITERATIONS)
# print("Final result:", result)

In [ ]:
@interact
def plot_thickness(id=widgets.Select(options=ids[:10]), aha_index=widgets.IntSlider(min=1, max=17, description="AHA segment")):
    t = np.load(f"notebooks/thicknesses/{id}_thickness_per_aha.npy")
    plt.scatter(x=range(50), y=t[:, aha_index-1])

In [ ]:
# count vertices per AHA segment 
# pd.Series(mesh.point_data['subpartID'].astype(int)).value_counts().sort_index()

In [ ]:
wall_thickness = pkl.load(open("data/transforms/cached/wall_thickness.pkl", "rb"))

In [ ]:
wall_thickness["wall_thickness_epicardial"].shape

In [ ]:
normalized_wt = []
for i, wt in enumerate(wall_thickness["wall_thickness_epicardial"]):
    cbrt_vol = wall_thickness["convex_hull_volumes"][i]**(1./3)
    normalized_wt.append(wt/cbrt_vol)
normalized_wt = np.array(normalized_wt)

In [ ]:
output_dir = "output"
experiments = [x for x in os.listdir(output_dir) if os.path.exists(os.path.join(output_dir, x, ".finished"))]

w = widgets.Dropdown(
    value="2020-09-11_02-13-41",
    options=experiments,
    description='Experiment:',
    disabled=False,
)

display(w)

In [ ]:
dic = pkl.load(open("data/transforms/cached/2ch_segmentation__LV__ED__non_scaled__dic.pkl", "rb"))

In [ ]:
# Get indices of each subpartition (LV endo and epi)
LVRV = Cardiac3DMesh("template/template.vtk").extractSubpart([1,2])

# Booleans indicating vertices that belong to each surface
endo_j = (LVRV.subpartID == 1)
epi_j = (LVRV.subpartID == 2)

In [ ]:
faces = np.hstack([[3] + list(x) for x in LVRV.triangles])
LVRV_pv = pv.PolyData(dic["mean"], faces)

In [ ]:
id = 2
# prepending a 3 before each triangle (PyVista format for faces)
faces_epi = np.hstack([[3] + list(x) for x in lv_epi.triangles])
lv_epi_pv = pv.PolyData(LVRV.points[epi_j], faces_epi)

faces_endo = np.hstack([[3] + list(x) for x in lv_endo.triangles])
lv_endo_pv = pv.PolyData(endo[id], faces_endo)

# lv_epi_pv.plot(scalars=wall_thickness[id])
# lv_epi_pv.rotate_z(180)
# lv_endo_pv.rotate_z(180)

In [ ]:
corr = spearmanr(z, normalized_wt, axis=0)

In [ ]:
def plot_mesh(mesh, faces, angle=0):
    
  surf = pv.PolyData(mesh, faces)
    
  surf.rotate_z(angle)
  plotter = pv.Plotter(notebook=True)
  # kargs = {"point_size": 2, "render_points_as_spheres": True}
    
  # surf.plot() #, **kargs)
  plotter.add_mesh(surf, show_edges=True)
  plotter.show(interactive=True)

  plotter.enable()

In [ ]:
def f(id, angle):
  
  lv_epi_pv = pv.PolyData(epi[id], faces_epi)
  lv_endo_pv = pv.PolyData(endo[id], faces_endo)

  lv_epi_pv.rotate_z(angle)
  lv_endo_pv.rotate_z(angle)
  
  plotter = pv.Plotter(notebook=False)
          
  # plotter.add_mesh(lv_epi_pv, opacity=0.8, scalars=wall_thickness['wall_thickness_epicardial'][id])
  plotter.add_mesh(lv_epi_pv, opacity=1, scalars=corr.correlation[8:,(1,)])
  plotter.add_mesh(lv_endo_pv, opacity=1)    
  plotter.show(interactive=True)
  plotter.enable()

interact(f, 
  id = widgets.SelectionSlider(options=range(200)),
  angle = widgets.SelectionSlider(options=range(360))
)

In [ ]:
def f():    
    plotter = pv.Plotter(notebook=True)          
    # plotter.add_mesh(LVRV_pv, opacity=0.8)#, scalars=wall_thickness['wall_thickness_epicardial'][id])
    plotter.add_mesh(lv_epi_pv, opacity=1, scalars=corr.correlation[8:,(1,)])
    plotter.add_mesh(lv_endo_pv, opacity=1)    
    plotter.show(interactive=True)
    plotter.enable()

f()